In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import json
# currently due to venv + VSCode issues, only runs in jupyter notebooks from command line

# Epic Games Store API wrapper from https://epicstore-api.readthedocs.io/en/latest/index.html
import epicstore_api
from epicstore_api import EpicGamesStoreAPI, OfferData
api = EpicGamesStoreAPI()

ModuleNotFoundError: No module named 'epicstore_api'

In [2]:
# grab initial table of all free epic games via scraping, saved as soup object
gameurl = "https://www.steamgifts.com/discussion/S4e2c/free-epic-games-store-list-of-all-weekly-free-games-every-thursday-at-11-am-et"
r = requests.get(gameurl)
r.status_code
gsoup = BeautifulSoup(r.text)
gtable = gsoup.find("table")

200

In [4]:

# proceed throughout rows and grab the text (could also have used pd.read_html)
table_rows = gtable.find_all('tr')
result = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        result.append(row)
gdf = pd.DataFrame(result, columns=["Number", "Name", "Start", "End"])
# gdf
# above code adapted from https://stackoverflow.com/questions/50633050/scrape-tables-into-dataframe-with-beautifulsoup

    Number                         Name       Start         End
0      477                   The Bridge   3/14/2024   3/21/2024
1      476     Deus Ex: Mankind Divided   3/14/2024   3/21/2024
2      475                 Astro Duel 2    3/7/2024   3/14/2024
3      474  Aerial_Knight’s Never Yield   2/29/2024    3/7/2024
4      473       Super Meat Boy Forever   2/22/2024   2/29/2024
..     ...                          ...         ...         ...
472      5                  Axiom Verge    2/7/2019   2/22/2019
473      4       The Jackbox Party Pack   1/24/2019    2/7/2019
474      3  What Remains of Edith Finch   1/11/2019   1/24/2019
475      2               Super Meat Boy  12/28/2018   1/10/2019
476      1                   Subnautica  12/12/2018  12/27/2018

[477 rows x 4 columns]


In [6]:
# grabbing links to Epic Games Store pages separately from same scraped table
links = []
for tr in table_rows:
    td = tr.find_all('td')
    for cell in td:
        link = cell.find('a')
        if link:
            links.append(link['href'].strip())

links_series = pd.Series(links)
# links_series
# above chunk adapted from chatGPT corrections of own orginal code

# tacking on to original df since shares index/order
gdf["Link"] = links_series
gdf

    Number                         Name       Start         End  \
0      477                   The Bridge   3/14/2024   3/21/2024   
1      476     Deus Ex: Mankind Divided   3/14/2024   3/21/2024   
2      475                 Astro Duel 2    3/7/2024   3/14/2024   
3      474  Aerial_Knight’s Never Yield   2/29/2024    3/7/2024   
4      473       Super Meat Boy Forever   2/22/2024   2/29/2024   
..     ...                          ...         ...         ...   
472      5                  Axiom Verge    2/7/2019   2/22/2019   
473      4       The Jackbox Party Pack   1/24/2019    2/7/2019   
474      3  What Remains of Edith Finch   1/11/2019   1/24/2019   
475      2               Super Meat Boy  12/28/2018   1/10/2019   
476      1                   Subnautica  12/12/2018  12/27/2018   

                                                  Link  
0       https://store.epicgames.com/en-US/p/the-bridge  
1    https://store.epicgames.com/en-US/p/deus-ex-ma...  
2    https://store.epicg

In [7]:
testurl = "https://store.epicgames.com/en-US/p/the-bridge"
rtest = requests.get(testurl)
rtest.status_code
# I think these pages are protected from scraping directly. Rather than use Selenium, on to APIs...

403

In [9]:
# the following will attempt to get just a single test entry, matched from search results

testgame = api.fetch_store_games(keywords = "The Bridge")
# testgame The Bridge, sometimes fails, but "Blue Oak Bridge" (not in dataset) always works as a tester

title_to_find = "Blue Oak Bridge"
matching_item = None

for element in testgame["data"]["Catalog"]["searchStore"]["elements"]:
    if element["title"] == title_to_find:
        matching_item = element
        break

if matching_item:
    print("Title:", matching_item["title"])
    print("Description:", matching_item["description"])
    # You can extract other fields as needed
else:
    print("Title not found.")

# print(matching_item)
print("Price:", matching_item["price"]["totalPrice"]["originalPrice"], "\n")

# PRETTY JSON PRINTING
def jprint(j):
    print(json.dumps(j, indent = 2, default = str))
jprint(matching_item)

Title: Blue Oak Bridge
Description: Blue Oak Bridge features breathtaking 2D hand-painted art in a storybook world of powerful witches, ancient magic and royal secrets. Farm, forage and fish throughout the island of Eloria and use all you discover to brew potions, make produce and help your new friends.
2499
{
  "title": "Blue Oak Bridge",
  "id": "aada95beab7c4671aa7cb3a6bb48c586",
  "namespace": "4dbe3446fbe1495a822cbea817f5606e",
  "description": "Blue Oak Bridge features breathtaking 2D hand-painted art in a storybook world of powerful witches, ancient magic and royal secrets. Farm, forage and fish throughout the island of Eloria and use all you discover to brew potions, make produce and help your new friends.",
  "effectiveDate": "2024-02-13T17:36:43.151Z",
  "keyImages": [
    {
      "type": "OfferImageWide",
      "url": "https://cdn1.epicgames.com/spt-assets/7faa14ae76ee4c15ae9d983ffc25224d/blue-oak-bridge-jd61u.jpg"
    },
    {
      "type": "OfferImageTall",
      "url": "h

In [16]:
# I now try to elaborate and extend the above approach with my own function

def get_gamedata(gamename, searchsize = 5):
    # takes a string gamename, searches for it (defaults to 5 search results), finds exact name/title match
    # returns dict with id, descr[iption], namespace, orig[inal]_price, fmt_orig_price (nicely formatted), and tags (list)
    dict = api.fetch_store_games(count = searchsize, keywords = gamename)
    match = None
    for element in dict["data"]["Catalog"]["searchStore"]["elements"]:
        if element["title"] == gamename:
            match = element
            break

    return_dict = {
    "id": match["id"],
    "descr": match["description"],
    "namespace": match["namespace"],
    "orig_price": match["price"]["totalPrice"]["originalPrice"],
    "fmt_orig_price": match["price"]["totalPrice"]["fmtPrice"]["originalPrice"],
    "tags": match["tags"]  # list but looks like individual dicts
    #TODO: possibly format the tags better
    #TODO: there might be one or two relevant pieces more to grab
    }
    return return_dict

    # alternative (decided against) where return is LIST type:
    # return [match["id"], match["description"], match["namespace"], match["price"]["totalPrice"]["originalPrice"],
    #         match["price"]["totalPrice"]["fmtPrice"]["originalPrice"]]

# I should mention that a few of the api calls might need other bits from the API JSON, but it's a bit unclear
# for example, slugs, namespace, various ID types all poorly defined and unclear when unique or extensible

# Let's test again with a single entry (same as previous)
jprint(get_gamedata("Blue Oak Bridge"))

{'id': 'aada95beab7c4671aa7cb3a6bb48c586', 'descr': 'Blue Oak Bridge features breathtaking 2D hand-painted art in a storybook world of powerful witches, ancient magic and royal secrets. Farm, forage and fish throughout the island of Eloria and use all you discover to brew potions, make produce and help your new friends.', 'namespace': '4dbe3446fbe1495a822cbea817f5606e', 'orig_price': 2499, 'fmt_orig_price': '$24.99', 'tags': [{'id': '1393'}, {'id': '1367'}, {'id': '1370'}, {'id': '9547'}, {'id': '9549'}, {'id': '1263'}]}


In [17]:
# example of weird tag formatting, currently list of dict-like elements
blueoak = get_gamedata("Blue Oak Bridge")
blueoak["tags"]
# see url for tag readable names https://epicstore-api.readthedocs.io/en/latest/models.html
#TODO: find a good way to include these tags in a more readable, cleaned way in final dataset for analysis

[{'id': '1393'},
 {'id': '1367'},
 {'id': '1370'},
 {'id': '9547'},
 {'id': '9549'},
 {'id': '1263'}]

In [34]:
# setting up a wrapped function to test other titles, eventually for the whole table, but to respect rate limits
def single_get(name, sleep_time = .5):
    time.sleep(sleep_time)
    #TODO: POTENTIALLY UNCOMMENT BELOW TEST OUTPUT PRINT STATEMENTS
    print("Attempting to grab data about", name)
    # No additional text = Success
    try:    
        done = get_gamedata(name)
    #TODO: trying to think of possibility get_gamedata fails... maybe add a better fail return?
    except:
        try:
            # in this case, possibly the name wasn't within the first 5 results, so trying more
            print("RETRY RETRY:", name)
            done = get_gamedata(name, 30)
        except:
            # on a fail, I will return None and I will have to make this into an empty row later
            print("\tFAILED RETRY")
            done = None
    return done

# testing on a small subset (to preserve time) of the table to make sure it works
# small_df = gdf.head()  # more reproducable
small_df = gdf.sample(5)  # better to tell if it works consistently
# small_df

small_df_result = [single_get(a_game) for a_game in small_df["Name"]]
small_df_result

Attempting to grab data about Sherlock Holmes: Crimes and Punishments
Attempting to grab data about Yoku’s Island Express
RETRY RETRY: Yoku’s Island Express
	FAILED RETRY
Attempting to grab data about Torchlight II
Attempting to grab data about Railway Empire
Attempting to grab data about What Remains of Edith Finch


[{'id': 'afadb715390a4593acdca79dae94b60a',
  'descr': 'Become the most celebrated detective of all time: Sherlock Holmes! Use your impressive talents as a detective to solve six thrilling and varied cases.',
  'namespace': '23f3da2016b94461a7bd7e2eb46aa0f9',
  'orig_price': 2999,
  'fmt_orig_price': '$29.99',
  'tags': [{'id': '21894'}, {'id': '9547'}, {'id': '1117'}]},
 None,
 {'id': '9ca2b214a7b0477cb57fb0975f50043c',
  'descr': 'Torchlight II is filled to the brim with randomized levels, enemies and loot. Capturing all the flavor and excitement of the original, Torchlight II expands the world and adds features players wanted most, including online and LAN multiplayer.',
  'namespace': '8432511ef63b489b843d21ff486a0355',
  'orig_price': 1999,
  'fmt_orig_price': '$19.99',
  'tags': [{'id': '1216'},
   {'id': '21122'},
   {'id': '1188'},
   {'id': '21894'},
   {'id': '21127'},
   {'id': '9547'},
   {'id': '1264'},
   {'id': '1265'},
   {'id': '21138'},
   {'id': '21139'},
   {'id': '

In [40]:
# Investigating some worrisome errors

# ff = gdf["Name"][3]
# get_gamedata("The Bridge")
# get_gamedata(ff)
api.fetch_store_games(count = 30, keywords = "Yoku’s Island Express") 

# Not working! Reason?
# AGE RESTRICTION, DUNNO HOW TO CIRCUMVENT
# perhaps I need to be logged in but wrapper doesn't have credential options
# fallback option: do Selenium after all (perhaps only for the failed, empty rows?)

{'data': {'Catalog': {'searchStore': {'elements': [],
    'paging': {'count': 30, 'total': 0}}}},
 'extensions': {}}

In [41]:
# CAUTION: TAKES A LONG TIME TO RUN! OUTPUTS A LOT TOO!
# This goes through the entire table and uses the API for all almost 500 games
# could add optional argument sleep_time for other than .5 if I want to though
big_df = [single_get(a_game) for a_game in gdf["Name"]]  # pulls all strings from "Name" column in gdf, same order
big_df

Attempting to grab data about The Bridge
RETRY RETRY: The Bridge
Attempting to grab data about Deus Ex: Mankind Divided
RETRY RETRY: Deus Ex: Mankind Divided
	FAILED RETRY
Attempting to grab data about Astro Duel 2
Attempting to grab data about Aerial_Knight’s Never Yield
RETRY RETRY: Aerial_Knight’s Never Yield
	FAILED RETRY
Attempting to grab data about Super Meat Boy Forever
Attempting to grab data about Dakar Desert Rally
Attempting to grab data about Lost Castle
Attempting to grab data about Doki Doki Literature Club Plus!
Attempting to grab data about Doors: Paradox
RETRY RETRY: Doors: Paradox
	FAILED RETRY
Attempting to grab data about Infinifactory
Attempting to grab data about Love
RETRY RETRY: Love
	FAILED RETRY
Attempting to grab data about Sail Forth
Attempting to grab data about Marvel’s Guardians of the Galaxy
RETRY RETRY: Marvel’s Guardians of the Galaxy
	FAILED RETRY
Attempting to grab data about A Plague Tale: Innocence
Attempting to grab data about 20 Minutes Till Daw

[{'id': '6b228a1ad940496dabc89ec6640107f3',
  'descr': 'The Bridge',
  'namespace': 'epic',
  'orig_price': 999,
  'fmt_orig_price': '$9.99',
  'tags': [{'id': '1298'},
   {'id': '1370'},
   {'id': '9547'},
   {'id': '1117'},
   {'id': '9549'}]},
 None,
 {'id': 'bc38eac0277d41ec955690e25779cb53',
  'descr': 'Astro Duel 2 is a sci-fi combat game combining top-down space dog fights with up-close platforming action. Whether versus or co-op, battle through space and on foot at the same time! Strike within the fully-destructible arenas or just nuke ‘em from orbit.',
  'namespace': '4a681a809a094e2c8dcc68353c68fed6',
  'orig_price': 1999,
  'fmt_orig_price': '$19.99',
  'tags': [{'id': '1216'},
   {'id': '21894'},
   {'id': '19847'},
   {'id': '9547'},
   {'id': '9549'},
   {'id': '1264'},
   {'id': '1299'},
   {'id': '1110'},
   {'id': '22776'},
   {'id': '1370'},
   {'id': '1151'},
   {'id': '1183'},
   {'id': '10719'}]},
 None,
 {'id': 'ef66029a1c0d467ea7755f6a88b4088c',
  'descr': 'Super

In [45]:
# worried about data loss (long time to re-run), saving copies two ways to not overwrite by accident
big_df_copy = big_df.copy()
big_df_copy2 = big_df

In [2]:
# first adding empty rows for all failed "None" games, then converting to dataframe
data = [d if d is not None else {} for d in big_df]
adf = pd.DataFrame(data)

# merging the API and scraped game data, indices should still match
# print(adf.columns, "\n", gdf.columns)
# print(adf.head(5), gdf.head(5))
joined = pd.merge(adf, gdf, left_index = True, right_index = True)
# print(joined.columns)
joined.sample(15)

# saving data to CSV
# adf.to_csv("~/Desktop/dshw/EGS_project/data/df_api.csv", index = True)  # A in adf for "A"pi source
# gdf.to_csv("~/Desktop/dshw/EGS_project/data/df_scraped.csv", index = True)  # G in gdf for "G"ames list source
joined.to_csv("~/Desktop/dshw/EGS_project/data/df_joined.csv", index = True)

NameError: name 'big_df' is not defined

In [ ]:
pd.read_csv()